In [19]:
# For Aston_Martin

from datetime import datetime
import pandas as pd
import re
import numpy as np

# Current year for the 'Brand New' cars
current_year = datetime.now().year

# Load the data
file_path = 'cleaned_auto_trader_data_aston_martin.csv' # Has to be the cleaned data as this is file that's updated
data = pd.read_csv(file_path)

# Function to extract numeric part from a string (for 'Price' and 'Mileage')
def extract_numeric(value):
    try:
        # Remove non-digit characters, preserve decimal points and negative signs
        numeric_string = re.sub("[^-0-9.]", "", str(value))
        return float(numeric_string)
    except ValueError:
        return np.nan  # Return NaN for non-numeric or empty values

# Clean the 'Year' column by removing parentheses and their contents
data['Year'] = data['Year'].astype(str).apply(lambda x: re.sub(r'\s*\(.*\)', '', x)).astype(float)

# Apply transformations and validations
data['Price'] = data['Price'].apply(extract_numeric)
data['Mileage'] = data['Mileage'].apply(extract_numeric)
data['Prior_Owners'] = data['Prior_Owners'].apply(extract_numeric)  # Updated to use extract_numeric

# Fill in missing data with 'Unknown' only for Engine_Size column
data['Engine_Size'] = data['Engine_Size'].fillna('Unknown')

# Fill in missing data with 'Unknown' for other categorical columns
categorical_columns = ['Model', 'Additional_Comments', 'Car_Type', 'Engine_Type', 'Dealership_Location', 'Sale_Type']
data[categorical_columns] = data[categorical_columns].fillna('Unknown')

# Remove rows with 'Unknown' values in critical columns
data = data[data['Price'] != 'Unknown']
data = data[data['Mileage'] != 'Unknown']
data = data[data['Year'] != 'Unknown']

# Adding the delete duplicates feature
cleaned_data = data.drop_duplicates()

# Save the cleaned data to a new file
cleaned_file_path = 'cleaned_auto_trader_data_aston_martin.csv'
data.to_csv(cleaned_file_path, index=False)

In [ ]:
# EXPLORATORY DATA ANALYSIS



In [20]:
# For Aston_Martin - Extended Data

from datetime import datetime
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import numpy as np
import joblib

# Load the data
print("Loading data...")
file_path = 'cleaned_auto_trader_data_aston_martin.csv'
data = pd.read_csv(file_path)

# Preprocess numeric columns
print("Preprocessing numeric columns...")
for col in ['Price', 'Mileage', 'Year', 'Prior_Owners']:
    data[col] = pd.to_numeric(data[col], errors='coerce')
    data[col].fillna(data[col].mean(), inplace=True)

# Preprocessing pipelines
print("Creating preprocessing pipelines...") # TODO visualise the data after and before preprocessing to see if any issues with it
numeric_features = ['Mileage', 'Year', 'Prior_Owners']
numeric_transformer = StandardScaler()

categorical_features = ['Make', 'Car_Type', 'Sale_Type']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# TF-IDF Vectorizer for 'Model' column
print("Applying TF-IDF Vectorization...") # TODO check results of this vectorisation. not sure TF-IDF is needed for "Model" of car tbh but interesting.
tfidf_vectorizer = TfidfVectorizer(max_features=2500)
X_tfidf = tfidf_vectorizer.fit_transform(data['Model'])

# Fit and transform the data using the preprocessor
X_preprocessed = preprocessor.fit_transform(data.drop('Model', axis=1)) # TODO check the data here is not a concatination, but just the preprocessed data 

# Combine TF-IDF features with preprocessed numeric and categorical features # TODO investigate how this data looks, both TFIDF and preprocessed, to see if we really can expect model to fit to both at same time
X_combined = np.hstack((X_tfidf.toarray(), X_preprocessed if isinstance(X_preprocessed, np.ndarray) else X_preprocessed.toarray()))
# TODO I think bunching this data together and using random forest isn't a good combination. Need to visualise both the TFIDF data and preprocessed data, seperately, and both before and after preprocessing. Do some exploratory data analysis. Visualise words with word map (bigger words for more frequent) and visualise numeric data with grid map different variables on each axis of grid

# Split the data
print("Splitting data...")
y = data['Price'].values
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# TODO try and get random forest to show which features it is weighting (importances)

# Model Training with Hyperparameter Tuning
print("Training model with hyperparameter tuning...")
model = RandomForestRegressor(random_state=42)
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}
grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Use the best estimator # TODO investigate the "performance" of each model with different hyperparamter tuning. Check it's actually finding a good one, not just the least poor performing.
best_model = grid_search.best_estimator_
print("Best model found:", best_model)

# Predictions and Evaluation
print("Making predictions and evaluating...")
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Output the model performance
print(f'MSE: {mse}')
print(f'R2 Score: {r2}') # TODO this score seems good. but not great. Investigate examples where it's accurate, and where it's messing up. see if there's anything obvious. back to EDA

# Save the model, vectorizer, and preprocessor
print("Saving model, vectorizer, and preprocessor...")
joblib.dump(best_model, 'car_price_prediction_model_aston_martin.joblib')
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer_aston_martin.joblib')
joblib.dump(preprocessor, 'preprocessor_aston_martin.joblib')

print("Process completed.")

Loading data...
Preprocessing numeric columns...
Creating preprocessing pipelines...
Applying TF-IDF Vectorization...
Splitting data...
Training model with hyperparameter tuning...
Best model found: RandomForestRegressor(max_depth=20, min_samples_split=10, n_estimators=50,
                      random_state=42)
Making predictions and evaluating...
MSE: 1250240567.3750648
R2 Score: 0.8530516286395321
Saving model, vectorizer, and preprocessor...
Process completed.


In [16]:
# Aston_Martin - Extended Data

import pandas as pd
import numpy as np
import joblib

# Load the saved model, vectorizer, and preprocessor
model = joblib.load('car_price_prediction_model_aston_martin.joblib')
vectorizer = joblib.load('tfidf_vectorizer_aston_martin.joblib')
preprocessor = joblib.load('preprocessor_aston_martin.joblib')

# Example new data for prediction
new_data = [
    {'Make': 'Aston Martin Vanquish', 'Model': '6.0 V12 Coupe 2dr Petrol T-TronicII Euro 5 (565 bhp)', 'Mileage': 33000, 'Year': 2013, 'Prior_Owners': '4', 'Car_Type': 'coupe', 'Sale_Type': 'Private seller'}
    # Add more cars as needed
]

# Convert to DataFrame
new_data_df = pd.DataFrame(new_data)

# Apply the preprocessing pipeline to the new data
X_new_tfidf = vectorizer.transform(new_data_df['Model'])
X_new_preprocessed = preprocessor.transform(new_data_df.drop('Model', axis=1))

# Convert to dense array if sparse
if hasattr(X_new_preprocessed, "toarray"):
    X_new_preprocessed = X_new_preprocessed.toarray()

# Combine TF-IDF features with preprocessed numeric and categorical features
X_new_combined = np.hstack((X_new_tfidf.toarray(), X_new_preprocessed))

# Make predictions
predicted_prices = model.predict(X_new_combined)

# Output predicted prices
for i, car in enumerate(new_data):
    print(f"Car: {car['Make']}, Predicted Price: £{predicted_prices[i]:.2f}")

Car: Aston Martin Vanquish, Predicted Price: £63102.17


In [24]:
import pandas as pd
import numpy as np
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

# Load the saved model, vectorizer, and preprocessor
model = joblib.load('car_price_prediction_model_aston_martin.joblib')
vectorizer = joblib.load('tfidf_vectorizer_aston_martin.joblib')
preprocessor = joblib.load('preprocessor_aston_martin.joblib')

# Load the cleaned dataset
data = pd.read_excel('cleaned_auto_trader_data_aston_martin.xlsx')

# Define preprocessing steps
numeric_features = ['Mileage', 'Year', 'Prior_Owners']
categorical_features = ['Make', 'Car_Type', 'Sale_Type']

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))]) #! this feels bad we don't want "missing" appearing in dataset, unless "missing" could actual devalue a car in dealers eyes in reality?

# Create a column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Preprocess numeric and categorical features
X_preprocessed = preprocessor.fit_transform(data)

# Preprocess text feature 'Model'
X_text = vectorizer.transform(data['Model'].fillna(''))

# Convert to dense array if sparse
if hasattr(X_preprocessed, "toarray"):
    X_preprocessed = X_preprocessed.toarray()
if hasattr(X_text, "toarray"):
    X_text = X_text.toarray()

# Combine the processed features
X_combined = np.hstack((X_preprocessed, X_text))

# Predict prices with the model
predicted_prices = model.predict(X_combined)

# Add the predicted prices to the DataFrame
data['Predicted_Price'] = predicted_prices

# Calculate the percentage difference between predicted and actual prices
data['Price_Diff_Percent'] = ((data['Predicted_Price'] - data['Price']) / data['Price']) * 100

# Filter to find potential undervalued cars with 10-35% price difference # TODO investigate the results in different brackets. Also investigate the distribution of these predictions (are there any outliers)
undervalued_cars = data[(data['Price_Diff_Percent'] > 10) & (data['Price_Diff_Percent'] < 35)]

# Sort the undervalued cars by Make and Location
sorted_undervalued_cars = undervalued_cars.sort_values(['Make', 'Dealership_Location'])

# Print out the sorted undervalued cars with 10-35% price difference, including Title_URL
print(sorted_undervalued_cars[['Make', 'Model', 'Dealership_Location', 'Price', 'Predicted_Price', 'Price_Diff_Percent', 'Title_URL', 'Sale_Type']])

sorted_undervalued_cars.to_csv('sorted_undervalued_cars.csv', index=False)


                     Make                                              Model  \
503      Aston Martin DB7                                    5.9 Vantage 2dr   
734      Aston Martin DB7                                            3.2 2dr   
636      Aston Martin DB7                                    5.9 Volante 2dr   
282      Aston Martin DB9                      6.0 V12 T-TronicII Euro 5 2dr   
856      Aston Martin DB9              6.0 V12 Volante T-TronicII Euro 5 2dr   
..                    ...                                                ...   
536  Aston Martin Vantage  LHD 4.3 V8 Coupe 2dr Petrol Obsidion Black Man...   
704  Aston Martin Vantage                                  4.7 V8 Euro 4 2dr   
789  Aston Martin Vantage                       4.7 V8 Sportshift Euro 4 2dr   
377   Aston Martin Virage                      6.0 V12 T-TronicII Euro 5 2dr   
94    Aston Martin Virage              6.0 V12 Volante T-TronicII Euro 5 2dr   

           Dealership_Location  Price  

In [38]:
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# Load the cleaned dataset, model, vectorizer, and preprocessor
data = pd.read_csv('cleaned_auto_trader_data_aston_martin.csv')
model = joblib.load('car_price_prediction_model_aston_martin.joblib')
vectorizer = joblib.load('tfidf_vectorizer_aston_martin.joblib')
preprocessor = joblib.load('preprocessor_aston_martin.joblib')

# Define the target car details
target_car = {
    'Make': 'Aston Martin', 
    'Model': 'Vantage 4.3 V8 Sportshift Euro 4 2dr', 
    'Mileage': 19000, 
    'Year': 2008, 
    'Prior_Owners': 1, 
    'Car_Type': 'Coupe', 
    'Sale_Type': 'Private seller'
}

# Convert the target car data into a DataFrame
target_car_df = pd.DataFrame([target_car], columns=[key for key in target_car.keys()])

# Process target car info for TF-IDF vectorization
target_car_df['Combined_Info'] = target_car_df['Make'] + " " + target_car_df['Model']
X_target_tfidf = vectorizer.transform(target_car_df['Combined_Info'])

# Drop the 'Make' and 'Model' columns if they were not used as raw features during training
# # Ensure other necessary columns are present
# target_car_df = target_car_df.drop(['Make', 'Model', 'Combined_Info'], axis=1)

# Transform the rest of the features using the preprocessor
X_target_preprocessed = preprocessor.transform(target_car_df)

# Combine TF-IDF features with preprocessed numeric and categorical features
X_target_combined = np.hstack((X_target_tfidf.toarray(), X_target_preprocessed.toarray()))

# Predict the price of the target car
predicted_price = model.predict(X_target_combined)
print(f"Predicted price of the entered car: {predicted_price[0]}")

similar_scores = []
for data in X_target_preprocessed:
    # Compute similarity scores
    similar_scores.append(cosine_similarity(data, X_target_tfidf))

print(similar_scores)
# Add similarity scores to the dataframe
data['Similarity'] = similar_scores.flatten()

# Sort the dataframe based on similarity
similar_cars = data.sort_values(by='Similarity', ascending=False)

# Display the top similar cars
print("Top similar cars:")
print(similar_cars[['Make', 'Model', 'Year', 'Price', 'Mileage', 'Car_Type', 'Sale_Type', 'Similarity']].head())

# Save the similar cars to an Excel file
similar_cars.to_excel('similar_cars.xlsx', index=False)




Predicted price of the entered car: 81023.855523803


ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 33 while Y.shape[1] == 110